In [1]:
import os
import requests
cred_url = os.environ["QCLOUD_CONTAINER_INSTANCE_CREDENTIALS_URL"]
r = requests.get(cred_url)
secretId = r.json()["TmpSecretId"]
secretKey = r.json()["TmpSecretKey"]
token = r.json()["Token"]

In [12]:
import os
from qcloud_cos import CosConfig
from qcloud_cos import CosS3Client
from ti.utils import get_temporary_secret_and_token

#### 指定本地文件路径，可根据需要修改。
local_file = "/home/tione/notebook/data/train_data.zip"

#### 用户的存储桶，修改为存放所需数据文件的存储桶，存储桶获取参考腾讯云对象存储
bucket="hc-02-1258788535"

#### 用户的数据，修改为对应的数据文件路径，文件路径获取参考腾讯云对象存储
data_key="BDC/train_data.zip"

#### 获取用户临时密钥
secret_id, secret_key, token = get_temporary_secret_and_token()
config = CosConfig(Region=os.environ.get('REGION'), SecretId=secret_id, SecretKey=secret_key, Token=token, Scheme='https')
client = CosS3Client(config)

####  获取文件到本地
response = client.get_object(
    Bucket=bucket,
    Key=data_key,
)
response['Body'].get_stream_to_file(local_file)

In [8]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm 
import datetime, time, gc 
from utils import distance, haversine, standard, pad_seq, geohash_encode
from scipy.stats import skew, kurtosis
from zipfile import ZipFile
from collections import Counter 
from sklearn.metrics import mean_squared_error as mse 
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
import collections 
from geopy import distance 

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [21]:
# !pip install geopy

In [2]:
# %%time

# myzip=ZipFile('data/train0711.zip')
# f=myzip.open('train0711.csv')
# train_df=pd.read_csv(f,  header=None)
# print(train_df.head())
# f.close()
# myzip.close() 

# print(train_df.shape)
# train_df = train_df.drop_duplicates().reset_index(drop=True)
# print(train_df.shape)
# train_df.columns=['loadingOrder', 'carrierName', 'timestamp', 'longitude', 'latitude', 'vesselMMSI', 'speed', 'direction', 'vesselNextport',
#                   'vesselNextportETA', 'vesselStatus', 'vesselDatasource', 'TRANSPORT_TRACE']
# train_df = train_df.drop(['vesselNextport', 'vesselNextportETA', 'vesselStatus', 'vesselDatasource'], axis=1)
# train_df.sort_values(['loadingOrder', 'timestamp'], inplace=True)

# train_df = train_df[train_df['TRANSPORT_TRACE'].notnull()] 

# train_df = train_df.reset_index(drop=True)

# print(train_df.shape)

# train_list = train_df.groupby('loadingOrder').agg({'longitude': list, 'latitude': list, 'speed': list,  'direction': list, 
#                                                   'carrierName': lambda x: x[0],
#                                                   'vesselMMSI': lambda x: x[0], 
#                                                   'timestamp': list,
#                                                   'TRANSPORT_TRACE': lambda x: x[0]}).reset_index()

In [3]:
# test=pd.read_csv('data/R2_ATest 0711.csv')
# print(test.shape) 
# test = test.drop_duplicates(['loadingOrder', 'timestamp'])
# print(test.shape)

# test = test.sort_values(['loadingOrder', 'timestamp']).reset_index(drop=True)

# test_list = test.groupby('loadingOrder').agg({'longitude': list, 'latitude': list, 'speed': list,  'direction': list, 
#                                       'carrierName': lambda x: x[0],
#                                       'vesselMMSI': lambda x: x[0], 
#                                       'timestamp': list,
#                                       'onboardDate': lambda x: x[0], 
#                                       'TRANSPORT_TRACE': lambda x: x[0]}).reset_index()


# test_train_trace_diff = set(test_list.TRANSPORT_TRACE.unique()) -set(train_list.TRANSPORT_TRACE.unique())

# test_trace_unique = set(test_list.TRANSPORT_TRACE.unique())
# train_trace_unique = set(train_list.TRANSPORT_TRACE.unique())

# trace_intersect = set(test_list.TRANSPORT_TRACE.unique()) & set(train_list.TRANSPORT_TRACE.unique())
# trace_other = set()
# for x1 in test_train_trace_diff:
#     print(x1)
#     for x in train_trace_unique:
#         if len(set(x.split('-')[:2]) & set(x1.split('-'))) == 2 and x not in trace_intersect:
#             trace_other.add(x)
            
# train_in = train_list[train_list['TRANSPORT_TRACE'].isin(trace_intersect)].reset_index(drop=True)
# train_out = train_list[train_list['TRANSPORT_TRACE'].isin(trace_other)].reset_index(drop=True)

# train_in.to_pickle("train_in_0718.pkl")
# train_out.to_pickle("train_out_0718.pkl")

In [4]:
train_in = pd.read_pickle("train_in_0718.pkl")
train_out = pd.read_pickle("train_out_0718.pkl")

In [5]:
test=pd.read_csv('data/R2_ATest 0711.csv')
print(test.shape) 
test = test.drop_duplicates(['loadingOrder', 'timestamp'])
print(test.shape)

test = test.sort_values(['loadingOrder', 'timestamp']).reset_index(drop=True)

test_list = test.groupby('loadingOrder').agg({'longitude': list, 'latitude': list, 'speed': list,  'direction': list, 
                                      'carrierName': lambda x: x[0],
                                      'vesselMMSI': lambda x: x[0], 
                                      'timestamp': list,
                                      'onboardDate': lambda x: x[0], 
                                      'TRANSPORT_TRACE': lambda x: x[0]}).reset_index()

(77654, 10)
(59386, 10)


In [10]:
ports_info=pd.read_csv('data/port.csv')

port_GPS_dict=collections.defaultdict(list)
for _,row in ports_info.iterrows():
    port_GPS_dict[row['TRANS_NODE_NAME']].append((row['LONGITUDE'],row['LATITUDE']))

In [13]:
port_GPS_dict['HON']

[(114.11075500000001, 22.335179)]

In [16]:
test_list.head(1)

,loadingOrder,longitude,latitude,speed,direction,carrierName,vesselMMSI,timestamp,onboardDate,TRANSPORT_TRACE
0,AC437723355280,"[113.895823, 113.917878, 113.97771200000001, 1...","[22.383463, 22.360128, 22.345253, 22.34531, 22...","[26, 27, 23, 23, 23, 23, 23, 23, 23, 20, 17, 1...","[16060, 11910, 8620, 9110, 9010, 9010, 9030, 8...",JCMFTA,G9916514058,"[2019-06-23T02:53:18.000Z, 2019-06-23T03:01:28...",2019/06/23 02:53:18,CNSHK-CLVAP


In [28]:
distance.distance((22.48627, 113.8621), (22.383463, 113.895823)).km

11.9020069873288

In [27]:
np.arange(30)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [43]:
for i in range(test_list.shape[0]):
    test_line = test_list.iloc[i]
    print(test_line)
    first_latlon = (test_line['latitude'][0], test_line['longitude'][0])  # lat lon
    
    last_latlon = (test_line['latitude'][-1], test_line['longitude'][-1])
    first_hash4 = geohash_encode(first_latlon[1], first_latlon[0], precision=4)
    last_hash4 = geohash_encode(last_latlon[1], last_latlon[0], precision=4)
    print(first_hash4, last_hash4)
    test_line_len = len(test_line['longitude'])
    
    for j in range(train_in.shape[0]):
        train_line = train_in.iloc[j]
        if train_line['TRANSPORT_TRACE'] != test_line['TRANSPORT_TRACE']:
            continue
        train_line_len = len(train_line['longitude'])   # 单条长度
        lat, lon = train_line['latitude'], train_line['longitude']
        line_latlon = [(lat[i], lon[i]) for i in range(train_line_len)]
        line_hash4 = [geohash_encode(lon[i], lat[i], precision=4) for i in range(train_line_len)]
        print(line_hash4[:10], line_hash4[-10:])
        print(line_hash4.index(first_hash4))
        print(line_hash4.index(last_hash4))
        print(line_hash4[::-1].index(last_hash4))
        break
        # first_dist = [distance.distance(latlon, first_latlon).km for latlon in line_latlon]
        
        #print(first_dist)

    break 

loadingOrder                                          AC437723355280
longitude          [113.895823, 113.917878, 113.97771200000001, 1...
latitude           [22.383463, 22.360128, 22.345253, 22.34531, 22...
speed              [26, 27, 23, 23, 23, 23, 23, 23, 23, 20, 17, 1...
direction          [16060, 11910, 8620, 9110, 9010, 9010, 9030, 8...
carrierName                                                   JCMFTA
vesselMMSI                                               G9916514058
timestamp          [2019-06-23T02:53:18.000Z, 2019-06-23T03:01:28...
onboardDate                                     2019/06/23  02:53:18
TRANSPORT_TRACE                                          CNSHK-CLVAP
Name: 0, dtype: object
webz rs5x
['webz', 'webz', 'webz', 'webz', 'webz', 'webz', 'webz', 'webz', 'webz', 'webz'] ['66jh', '66jh', '66jh', '66jh', '66jh', '66jh', '66jh', '66jh', '66jh', '66jh']
0
8945
33467


In [42]:
test_line_len, train_line_len

(334, 42477)

In [39]:
test_list[test_list['TRANSPORT_TRACE'] == 'HUNGARY-HON']

,loadingOrder,longitude,latitude,speed,direction,carrierName,vesselMMSI,timestamp,onboardDate,TRANSPORT_TRACE
45,GD501404074416,"[14.189217000000001, 14.454867000000002, 14.45...","[44.974016999999996, 45.31955, 45.319483, 45.3...","[32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[2000, 4800, 1700, 32100, 22400, 400, 21400, 2...",RWHZVZ,T1268450013,"[2019-02-26T11:42:08.000Z, 2019-02-26T17:52:38...",2019/02/26 11:42:08,HUNGARY-HON


In [7]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          96166       26443       51604          18       18117       69559
Swap:             0           0           0
